# DGraph QA chain

This notebook shows how to use LLMs to provide a natural language interface to a [DGraph](https://dgraph.io/) database.

# Running a DGraph instance

### Local Installation
You can get a local docker instance running with the DGraph Docker Image:

```
docker run --name dgraph -d -p "8080:8080" -p "9080:9080" -v <DATA_PATH>:/dgraph dgraph/standalone:latest
```
Where <DATA_PATH> is a directory on your local machine where you want to store the DGraph data.
Example:
```
docker run --name dgraph -d -p "8080:8080" -p "9080:9080" -v C:/dgraph dgraph/standalone:latest
```

### Cloud Installation
Alternatively you can run a DGraph instance on the cloud. For example, you can use the [DGraph Cloud](https://cloud.dgraph.io/pricing?type=free&zo=us-east-1) service.


# Instantiate a DGraph Object

In [ ]:
# Install pydgraph dependency
pip install pydgraph

In [1]:
from langchain.graphs import DGraph
graph = DGraph(clientUrl="localhost:9080")
# if you have a namespace in your local instance
#graph = DGraph(clientUrl="localhost:8080", namespace="NAMESPACE", username="USERNAME", password="PASSWORD")

# Or if you have a cloud instance
#graph = DGraph(clientUrl="YOUR_CLOUD_INSTANCE_URL", apiToken="API_TOKEN")

# If you have namespaces in your cloud instance
#graph = DGraph(clientUrl="CLIENT_URL", apiToken="API_TOKEN", namespace="NAMESPACE", username="USERNAME", password="PASSWORD")

In [3]:
# Some sample data into the DGraph instance
schema = """
<age>: int .
<cast>: [uid] @reverse .
<name>: string @index(hash) .
<year>: string @index(exact) .
type <Movie> {
	cast
	name
	year
}
type <Actor> {
	name
	age
}
"""

actorsRDF = """
_:tom <name> "Tom Hanks" .
_:tom <age> "63" .
_:tom <dgraph.type> "Actor" .

_:jennifer <name> "Jennifer Lawrence" .
_:jennifer <age> "33" .
_:jennifer <dgraph.type> "Actor" .


_:porom <name> "Porom Kamal" .
_:porom <age> "21" .
_:porom <dgraph.type> "Actor" .

_:forrest <name> "Forrest Gump" .
_:forrest <year> "1994" .
_:forrest <dgraph.type> "Movie" .
_:forrest <cast> _:tom .

_:terminal <name> "The Terminal" .
_:terminal <year> "2004" .
_:terminal <dgraph.type> "Movie" .
_:terminal <cast> _:tom .
_:terminal <cast> _:porom .

_:hunger <name> "The Hunger Games" .
_:hunger <year> "2012" .
_:hunger <dgraph.type> "Movie" .
_:hunger <cast> _:jennifer .
"""

In [4]:
graph.drop_all()
graph.add_schema(schema)
graph.add_node_rdf(actorsRDF)

# Instantiate a DGraphQAChain Object

In [8]:
from langchain.chains import DGraphQAChain
from langchain.chat_models import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

chain = DGraphQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [ ]:
chain.run("How old is Tom Hanks?")

In [ ]:
chain.run("Who acted in the movie The Terminal?")

In [ ]:
chain.run("Did Jennifer Lawrence act in a movie that was released in 2012?")

## Chain Modifiers
You can alter the values of the following `DGraphQAChain` class variables to modify the behaviour of your chain results


In [ ]:
# Specify the maximum amount of DQL Generation attempts that should be made
chain.max_generation_attempts = 5

# Specify a set of DQL Query Examples, which are passed to
# the DQL Generation Prompt Template to promote few-shot-learning.
# Defaults to an empty string.
chain.dql_examples = """
Question: Find all movies that were released after 2016
DQL Query: 
{
  find_movies(func: type(Movie)) @filter(gt(year, "2016")){
    expand(_all_)
  }
}
"""

# Specify whether to use a list of generic DQL Query Examples
chain.use_generic_query_examples = True

# Enable experimental DQL Syntax Injection, which inserts DQL Syntax notes
# into the DQL Generation Prompt Template.
chain.experimental_dql_syntax_injection = True